### Introduction:

Alot of raw data is available in <b>CSV, JSON, XML, and other “standard” formats</b>, an awful lot of published data resides within webpages, embedded within HTML markup (usually within <b>"table"</b> tags, but not always). Developing strategies and techniques for extracting usable data from raw HTML is an important skill, and this assignment helps us successfully extract ,<b>“web data”</b> using python techniques and modeling the data in a clean format using techniques such as <b>BeautifulSoup</b> and storing the extracted data in a database using <b>DBcm</b> a Python embedded SQL technique to connect to a database using Python and derive beautiful and meaningful insights related to data and answer some question or business problem in real time.

### Please install/import below mentioned modules

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.io.html import read_html
import bs4 as BeautifulSoup
import urllib.request as u
import DBcm

### Let's extract the table we have to work on from below link:

In [2]:
webpage = "https://en.wikipedia.org/wiki/Taoiseach"

In [3]:
tables = read_html(webpage, attrs = {'class':'wikitable'})

In [4]:
print("Found {num} wikitables".format(num = len(tables)))

Found 1 wikitables


### The extracted dataframe table is stored in a variable name df:

In [5]:
df = tables[0]

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit],President of the Executive Council[edit]
1,No.,Portrait,Name(Birth–Death)Constituency,Term of office,Term of office,Party,Exec. CouncilComposition,Exec. CouncilComposition,Vice President,Vice President,Dáil(elected)
2,1,NaN,W. T. Cosgrave(1880–1965)TD for Carlow–Kilkenn...,6 December1922[note 10],9 March1932,Sinn Féin(Pro-Treaty),1st,SF (PT) (minority),NaN,Kevin O'Higgins,3 (1922)
3,1,NaN,W. T. Cosgrave(1880–1965)TD for Carlow–Kilkenn...,6 December1922[note 10],9 March1932,Cumann na nGaedheal,2nd,CnG (minority),NaN,Kevin O'Higgins,4 (1923)
4,1,NaN,W. T. Cosgrave(1880–1965)TD for Carlow–Kilkenn...,6 December1922[note 10],9 March1932,Cumann na nGaedheal,3rd,CnG (minority),NaN,Ernest Blythe,5 (Jun.1927)


### Now, let's drop some unwanted rows and columns 

In [7]:
df = df.drop(df.index[[10,11]])

In [8]:
df = df.drop(df.index[0])

In [9]:
df = df.drop([0], axis="columns")

In [10]:
df = df.drop([1], axis="columns")

In [11]:
df = df.drop([6], axis="columns")

In [12]:
df = df.drop([7], axis="columns")

In [13]:
df = df.drop([8], axis="columns")

### Let's split data from columns and separate them in different columns

In [14]:
df_name = df[2].str.split('(',expand = True)[0]

In [15]:
df_conc1 = df[2].str.split('for ',expand = True)[1].str.split('until',expand = True)[0].str.strip(' ')

In [16]:
df_conc2 = df[2].str.split('for ', expand = True)[2].str.split('from', expand = True)[0].str.strip(' ')

In [17]:
df_term_start = df[3].str.split('[', expand = True)[0]

In [18]:
df_term_end = df[4].str.split('[', expand = True)[0]

In [19]:
df_dail = df[10].str.split('(', expand = True)[0]

In [20]:
df = pd.concat([df, df_name, df_conc1, df_conc2, df_term_start, df_term_end,df_dail], axis = "columns")

In [21]:
df = df.drop([2], axis="columns")

In [22]:
df = df.drop([3], axis="columns")

In [23]:
df = df.drop([4], axis="columns")

In [24]:
df = df.drop([10], axis="columns")

In [25]:
df.columns = ['Party', 'Vice President', 'Name', 'Constituency 1', 'Constituency 2', 'Office Term Start', 'Office Term End', 'No. of times Elected']

In [26]:
df.loc[df['Office Term End'] == "Incumbent", 'Office Term End'] = datetime.today()

In [27]:
df = df.drop([1], axis = 0)

In [28]:
df = df[['Name', 'Constituency 1', 'Constituency 2', 'Office Term Start', 'Office Term End', 'Party', 'Vice President', 'No. of times Elected']]

In [29]:
df['Office Term Start'] = pd.to_datetime(df['Office Term Start'], errors = 'coerce')

In [30]:
df['Office Term End'] = pd.to_datetime(df['Office Term End'], errors = 'coerce')

### Here's our clean and tidy data with birth date missing, lets extract them in belo steps:

In [31]:
df.head()

,Name,Constituency 1,Constituency 2,Office Term Start,Office Term End,Party,Vice President,No. of times Elected
2,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Sinn Féin(Pro-Treaty),Kevin O'Higgins,3
3,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Kevin O'Higgins,4
4,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Ernest Blythe,5
5,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Ernest Blythe,6
6,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Ernest Blythe,6


In [32]:
df['No. of times Elected'] = pd.to_numeric(df['No. of times Elected'], errors='coerce')

### Extracting Birth dates for each Taoiseach

In [33]:
df.head()

,Name,Constituency 1,Constituency 2,Office Term Start,Office Term End,Party,Vice President,No. of times Elected
2,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Sinn Féin(Pro-Treaty),Kevin O'Higgins,3
3,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Kevin O'Higgins,4
4,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Ernest Blythe,5
5,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Ernest Blythe,6
6,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09,Cumann na nGaedheal,Ernest Blythe,6


In [34]:
names = df["Name"].str.replace(" ","_").str.replace("É","E").str.replace("á","a").values

In [35]:
names

array(['W._T._Cosgrave', 'W._T._Cosgrave', 'W._T._Cosgrave',
       'W._T._Cosgrave', 'W._T._Cosgrave', 'Eamon_de_Valera',
       'Eamon_de_Valera', 'Eamon_de_Valera', 'Eamon_de_Valera',
       'Eamon_de_Valera', 'Eamon_de_Valera', 'Eamon_de_Valera',
       'John_A._Costello', 'Eamon_de_Valera', 'John_A._Costello',
       'Eamon_de_Valera', 'Sean_Lemass', 'Sean_Lemass', 'Sean_Lemass',
       'Jack_Lynch', 'Jack_Lynch', 'Liam_Cosgrave', 'Jack_Lynch',
       'Charles_Haughey', 'Garret_FitzGerald', 'Charles_Haughey',
       'Garret_FitzGerald', 'Garret_FitzGerald', 'Charles_Haughey',
       'Charles_Haughey', 'Charles_Haughey', 'Albert_Reynolds',
       'Albert_Reynolds', 'Albert_Reynolds', 'John_Bruton',
       'Bertie_Ahern', 'Bertie_Ahern', 'Bertie_Ahern', 'Bertie_Ahern',
       'Brian_Cowen', 'Enda_Kenny', 'Enda_Kenny', 'Enda_Kenny',
       'Leo_Varadkar', 'Leo_Varadkar', 'Micheal_Martin'], dtype=object)

In [36]:
date_list = []

#### Here we are going to extract the birth-date of every president using beautiful-soup technique in python and store it in the above created empty list. The purpose of doing so is to get the age of the members of Taoiseach  at which age each of them assumed the office.

In [37]:
for i in range(0,45):
    urls = 'https://en.wikipedia.org/wiki/'+names[i]
    page = u.urlopen(urls)
    soup = BeautifulSoup.BeautifulSoup(page, 'lxml')
    var = soup.find('table',attrs={'class':'infobox vcard'})
    bd_day = var.find('span',attrs = {'class':'bday'})
    date_list.append(bd_day.text)

#### The birth dates are extracted for each Taoiseach member and stored in the date_list, let's convert the list to a dataframe so we can concat it to our main dataframe.

In [38]:
dates = pd.DataFrame(date_list) 

In [39]:
dates.columns = ["Birth_Dates"]

In [40]:
df = df.reset_index(drop=True)

In [41]:
df = pd.concat([df,dates], axis = "columns")

In [42]:
df = df.fillna('  ')

### Final Data with all needed columns and information

In [43]:
df['Birth_Dates'] = pd.to_datetime(df['Birth_Dates'], errors = 'coerce')
df

,Name,Constituency 1,Constituency 2,Office Term Start,Office Term End,Party,Vice President,No. of times Elected,Birth_Dates
0,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09 00:00:00.000000,Sinn Féin(Pro-Treaty),Kevin O'Higgins,3,1880-06-06
1,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09 00:00:00.000000,Cumann na nGaedheal,Kevin O'Higgins,4,1880-06-06
2,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09 00:00:00.000000,Cumann na nGaedheal,Ernest Blythe,5,1880-06-06
3,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09 00:00:00.000000,Cumann na nGaedheal,Ernest Blythe,6,1880-06-06
4,W. T. Cosgrave,Carlow–Kilkenny,Cork Borough,1922-12-06,1932-03-09 00:00:00.000000,Cumann na nGaedheal,Ernest Blythe,6,1880-06-06
5,Éamon de Valera,Clare,,1932-03-09,1937-12-29 00:00:00.000000,Fianna Fáil,Seán T. O'Kelly,7,1882-10-14
6,Éamon de Valera,Clare,,1932-03-09,1937-12-29 00:00:00.000000,Fianna Fáil,Seán T. O'Kelly,8,1882-10-14
7,Éamon de Valera,Clare,,1932-03-09,1937-12-29 00:00:00.000000,Fianna Fáil,Seán T. O'Kelly,9,1882-10-14
8,Éamon de Valera,Clare,,1937-12-29,1948-02-18 00:00:00.000000,Fianna Fáil,Seán T. O'Kelly,9,1882-10-14
9,Éamon de Valera,Clare,,1937-12-29,1948-02-18 00:00:00.000000,Fianna Fáil,Seán T. O'Kelly,10,1882-10-14


### Converting DataFrame to a list so we can insert it in the SQL Table

In [44]:
final_data = df.values.tolist()

### Inserting the data into out database table

In [45]:
config = {
    'user': 'Saurabh',
    'password': 'Saurabh@06',
    'host': 'localhost',
    'database': 'Ireland_Presidents',
}

#### Once we have created our database in SQL with above credentials we can insert the dataframe data into a table using DBcm package in python as below. But for it to insert properly we first need to create a database and a table with the right schema in SQL in following below steps.

In [ ]:
with DBcm.UseDatabase(config) as cursor:
    SQL = "insert into Taoiseach (Name, Constituency_1, Constituency_2, Office_Term_Start, Office_Term_End, Party, Vice_President, No_of_Times_Elected, Date_of_Birth) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.executemany(SQL, final_data)

### Table Schema & Steps

#### Once the dataframe has been inserted in our SQL database and desired table, we are ready to explore the data with some insights after studying the data and answer some questions based on it as well of which some examples are explained below.

#### Q.3. (a) Provide a list of constituencies in Ireland which have produced a Taoiseach.

#### Q.3. (b) Which political party has produced the most individual Taoiseachs?

SELECT Party,COUNT(*) FROM Taoiseach GROUP BY Party order by count(*) desc limit 1;

#### Q.3. (c) Provide the list of names for those politicians who previously held the position of “Vice President” or “Tánaiste” prior to becoming Taoiseach.

#### Q.3. (d) Which political party held the office of Taoiseach for the longest amount of overall time, and for how long (in years, months, and days)? (Answer is only in days :-( )

#### Q.3 (e) Which individual politician held the office of Taoiseach for the longest amount of uninterrupted time, and for how long (in years, months, and days)? (Answer is only in days :-( )

#### Q.3. (f) Which individual politician held the office of Taoiseach for the shortest amount of time, and for how long (in years, months, and days)? (Answer is only in days :-( )

#### Q.3. (g) Which political party has held the office of Taoiseach for the largest number of Dáils?

#### Q.3. (h) Which individual politician has held the office of Taoiseach for the most number of Dáils?

#### Q.3. (i) What age was each Taoiseach on the day they assumed office?

#### Q.3. (j) On the last day of their term of office, which politician was the oldest?

#### Q.3. (k) In your view, and based on statistics calculated from your scraped data, which Irish political party is the most successful? Show and describe your calculations.

### Conclusion

We have succssfully, created a automated python script that extracts <b>"web-data"</b> and store it in a database using python embedded package and help answer some questions listed using the extracted data.
The skill of extracting usable data from raw HTML is an important skill, which in real-time helps data analyst extract the information and help answer business their business problems.

Thank's for your valuable time to read and review the work and I'am open to any suggestions and feedbacks realted to my work.